In [1]:
import pandas as pd
from datetime import datetime
import json
import random

import folium
import branca.colormap as cmp


In [4]:
df = pd.read_pickle("data/W_B_df.pkl")
df = df[ (df.origin == 1) & ( df.dow == datetime.today().isoweekday())  ]
df = df.sort_values('destiny')
df = df.reset_index(drop=True)

df

,origin,destiny,dow,mean_time_s,std_time_s
0,1,1,5,272,253
1,1,2,5,293,239
2,1,3,5,551,262
3,1,4,5,547,278
4,1,5,5,258,183
...,...,...,...,...,...
124,1,127,5,1373,413
125,1,128,5,1407,383
126,1,129,5,1499,377
127,1,130,5,1626,410


In [6]:
tiempo_maximo = 15

In [8]:
geo_json = json.load( open ("data/madrid_barrios.geojson")          )
areas = {}
for id in range(len(geo_json["features"])) :
    cod = int(geo_json["features"][id]["properties"]["MOVEMENT_ID"])
    name = geo_json["features"][id]["properties"]["DISPLAY_NAME"]
    areas[cod] = name
df_areas = pd.DataFrame.from_dict(areas,orient="index")
df_areas.columns= ["area"]
df_areas["mean_time_m"] = 1000
for i,row in df.iterrows() :
    df_areas.loc[ [row["destiny"]], ["mean_time_m"]] = row["mean_time_s"]/60
df_areas.tail(20)

,area,mean_time_m
112,181 Casco Hist�rico de Vallecas,15.283333
113,182 Santa Eugenia,16.666667
114,183 Ensanche de Vallecas,18.050000
115,191 Casco Hist�rico de Vic�lvaro,17.333333
116,192 Valdebernardo,17.133333
117,193 Valderrivas,1000.000000
118,194 El Ca�averal,24.216667
119,201 Simancas,22.166667
120,202 Hell�n,1000.000000
121,203 Amposta,19.600000


In [9]:
valores_dict = df_areas.set_index('area')['mean_time_m']
valores_dict

area
011 Palacio                        4.533333
012 Embajadores                    4.883333
013 Cortes                         9.183333
014 Justicia                       9.116667
015 Universidad                    4.300000
                                    ...    
211 Alameda de Osuna              22.883333
212 Aeropuerto                    23.450000
213 Casco Hist�rico de Barajas    24.983333
214 Tim�n                         27.100000
215 Corralejos                    24.633333
Name: mean_time_m, Length: 131, dtype: float64

In [10]:
valores_dict.max()

1000.0

In [11]:
max(df["mean_time_s"]/60)

27.1

In [16]:
vmin = valores_dict.values.min()
#vmax = max(df["mean_time_s"]/60)
vmax = tiempo_maximo

step = cmp.StepColormap(['green','yellow', 'red'], 
                        vmin=vmin, 
                        vmax=vmax, 
                        index=[ vmin , vmin+0.7*(vmax-vmin), vmin+0.95*(vmax-vmin), vmax ],  #for change in the colors, not used fr linear
                        caption='Color Scale for Map'    #Caption for Color scale or Legend 
                       )
step

In [18]:
mapa = folium.Map(location=[40.416775, -3.703790], zoom_start=11, tiles="cartodbpositron")

#folium.GeoJson(geo_json ).add_to(mapa)

style_function = lambda feature: {'fillColor': step(valores_dict[feature["properties"]['DISPLAY_NAME']]) }

folium.GeoJson(geo_json, style_function=style_function ).add_to(mapa)

mapa